In [126]:
from lab_tsp_insertion import *
import math

In [127]:
class TSPSolution(object):
    def __init__(self, instance: list, tour:list, distance:int):
        self.tour = tour
        self.n_cities = len(tour)
        self.inst = instance
        self.distance = self.calculate_solution_distance()
    
    def __repr__(self):
        return f"Tour={self.tour}, distance={self.distance}, n_cities={self.n_cities}"
    
    def get_n_cities(self):
        return self.n_cities
    
    def get_distance(self, x, y):
        # This function takes the city number, not index into array
        c1x, c1y = self.inst[x]
        c2x, c2y = self.inst[y]
        return math.sqrt((c2x - c1x) ** 2 + (c2y - c1y) ** 2)
    
    def copy(self):
        return TSPSolution(self.inst, self.tour.copy(), self.distance)
    
    def is_valid_swap(self, x:int, y:int):
        x, y = min(x, y), max(x, y)
        x1 = (x + 1) % self.n_cities
        y1 = (y + 1) % self.n_cities
        return x != y and x != y1 and x != x1 and y != y1 and y != x1 and y1 != x1
    
    def calculate_cost_if_swapped(self, x:int, y: int):
        # If the edges are swapped what would be the cost
        x, y = min(x, y), max(x, y)
        x1 = (x + 1) % self.n_cities
        y1 = (y + 1) % self.n_cities
        xx1 = self.get_distance(self.tour[x], self.tour[x1])
        xy = self.get_distance(self.tour[x], self.tour[y])
        yy1 = self.get_distance(self.tour[x], self.tour[y1])
        x1y1 = self.get_distance(self.tour[x1], self.tour[y1])
        return self.distance - xx1 - yy1 + x1y1 + xy
    
    def calculate_solution_distance(self):
        dist = 0
        for x in range(self.n_cities):
            y = (x + 1) % self.n_cities
            dist += self.get_distance(self.tour[x], self.tour[y])
        return dist
    
    def perform_swap(self, x, y):
        x, y = min(x, y), max(x, y)
        new_distance = self.calculate_cost_if_swapped(self, x, y)
        x1 = (x + 1) % self.n_cities
        y1 = (y + 1) % self.n_cities
        self.tour[x1], self.tour[y] = self.tour[y], self.tour[x1]
        i = x1 + 1
        j = y - 1
        while (i < j):
            self.tour[i], self.tour[j] = self.tour[j], self.tour[i]
            i += 1
            j -= 1
        if self.calculate_solution_distance() != new_distance:
            print(f"New distance should have been {new_distance} but is {self.calculate_solution_distance()}")
        self.distance = new_distance


In [143]:
class TSPHillClimbing(object):
    def __init__(self, inst:dict = None):
        self.inst = inst
        self.ind = None
        self.g_best_solution = None
        self.g_best_distance = 99999999999999999999999999999999999999999
        if None != self.inst:
            self.ind = self.get_solution()
            self.update_best_g_instance(self.ind)
            
    def __repr__(self):
        return f"inst = {self.inst}\n\nindividual = {self.individual}\n\n" +\
             f"g_best_solution = {self.g_best_solution}\n\ng_best_distance = {self.g_best_distance}"
    
    def update_best_g_instance(self, instance: TSPSolution):
        self.g_best_solution = instance.copy()
        self.g_best_distance = instance.distance
        
    def get_solution(self)->Individual:
        cities, distance = insertion_heuristic1(self.inst)
        return TSPSolution(self.inst, cities, distance)
    
    def check_improving_move(self):
        current_cost = self.ind.distance
        for i in range(self.ind.get_n_cities()):
            for j in range(i+2, self.ind.get_n_cities()):
                if not self.ind.is_valid_swap(i, j):
                    print(f"{i}, {j} is not a valid swap")
                else:
                    newcost = self.ind.calculate_cost_if_swapped(i, j)
                    if newcost < current_cost:
                        print(f"Improved swapping {i}, {j} : {current_cost} -> {newcost}")

In [144]:
def main():
    inst = readInstance('small/inst-0.tsp')
    tsp = TSPHillClimbing(inst)
    tsp.check_improving_move()
    return tsp

In [145]:
tsp = main()

Improved swapping 0, 2 : 1653687.2696591003 -> 1649990.669924764
0, 38 is not a valid swap
Improved swapping 2, 4 : 1653687.2696591003 -> 1633463.4666229817
Improved swapping 4, 6 : 1653687.2696591003 -> 1631521.6913486721
Improved swapping 4, 7 : 1653687.2696591003 -> 1649172.2090688357
Improved swapping 4, 8 : 1653687.2696591003 -> 1632835.239084849
Improved swapping 5, 8 : 1653687.2696591003 -> 1647272.671726965
Improved swapping 6, 8 : 1653687.2696591003 -> 1649134.0055277469
Improved swapping 9, 11 : 1653687.2696591003 -> 1626275.26151856
Improved swapping 9, 12 : 1653687.2696591003 -> 1628617.5133399111
Improved swapping 9, 13 : 1653687.2696591003 -> 1640809.5415758342
Improved swapping 9, 14 : 1653687.2696591003 -> 1652819.022354936
Improved swapping 10, 12 : 1653687.2696591003 -> 1630302.0811738628
Improved swapping 10, 13 : 1653687.2696591003 -> 1643367.535655375
Improved swapping 10, 14 : 1653687.2696591003 -> 1653035.3321384424
Improved swapping 14, 16 : 1653687.2696591003 -

In [ ]:
print(tsp)

In [ ]:
8 % 8